# STEP 4: Count the number of observations in each ecosystem, during

each month of 2023

Much of the data in GBIF is **crowd-sourced**. As a result, we need not
just the number of observations in each ecosystem each month – we need
to **normalize** by some measure of **sampling effort**. After all, we
wouldn’t expect the same number of observations in the Arctic as we
would in a National Park, even if there were the same number of Veeries.
In this case, we’re normalizing using the average number of observations
for each ecosystem and each month. This should help control for the
number of active observers in each location and time of year.

### Set up your analysis

First things first – let’s load your stored variables.

In [1]:
%store -r

In [2]:
eco_gdf.columns

Index(['OBJECTID', 'ECO_NAME', 'BIOME_NUM', 'BIOME_NAME', 'REALM',
       'ECO_BIOME_', 'NNH', 'ECO_ID', 'SHAPE_LENG', 'SHAPE_AREA', 'NNH_NAME',
       'COLOR', 'COLOR_BIO', 'COLOR_NNH', 'LICENSE', 'geometry'],
      dtype='object')

### Identify the ecoregion for each observation

You can combine the ecoregions and the observations **spatially** using
a method called `.sjoin()`, which stands for spatial join.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-read"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Read More</div></div><div class="callout-body-container callout-body"><p>Check out the <a
href="https://geopandas.org/en/stable/docs/user_guide/mergingdata.html#spatial-joins"><code>geopandas</code>
documentation on spatial joins</a> to help you figure this one out. You
can also ask your favorite LLM (Large-Language Model, like ChatGPT)</p></div></div>

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Perform a spatial join</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Identify the correct values for the <code>how=</code> and
<code>predicate=</code> parameters of the spatial join.</li>
<li>Select only the columns you will need for your plot.</li>
<li>Run the code.</li>
</ol></div></div>

In [3]:
gbif_ecoregion_gdf = (
    eco_gdf
    # Match the CRS of the GBIF data and the ecoregions
    .to_crs(species_gbif_gdf.crs)
    # Find ecoregion for each observation
    .sjoin(
        species_gbif_gdf,
        how='inner', 
        predicate='contains')
    # Select the required columns
        [['gbifID','OBJECTID','month','ECO_NAME']]
        # video deleted this .reset_index()
        .rename(columns={
            'OBJECTID': 'ecoregion',
            'gbifID': 'observation_id'})
)
gbif_ecoregion_gdf

,observation_id,ecoregion,month,ECO_NAME
ecoregion,,,,
12,4743927038,13.0,5,Alberta-British Columbia foothills forests
12,4621947377,13.0,5,Alberta-British Columbia foothills forests
12,4761090115,13.0,6,Alberta-British Columbia foothills forests
12,4765238615,13.0,7,Alberta-British Columbia foothills forests
12,4630693711,13.0,6,Alberta-British Columbia foothills forests
...,...,...,...,...
839,4633848077,845.0,10,North Atlantic moist mixed forests
839,4749131402,845.0,9,North Atlantic moist mixed forests
839,4763942306,845.0,9,North Atlantic moist mixed forests


### Count the observations in each ecoregion each month

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Group observations by ecoregion</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Replace <code>columns_to_group_by</code> with a list of columns.
Keep in mind that you will end up with one row for each group – you want
to count the observations in each ecoregion by month.</li>
<li>Select only month/ecosystem combinations that have more than one
occurrence recorded, since a single occurrence could be an error.</li>
<li>Use the <code>.groupby()</code> and <code>.mean()</code> methods to
compute the mean occurrences by ecoregion and by month.</li>
<li>Run the code – it will normalize the number of occurrences by month
and ecoretion.</li>
</ol></div></div>

In [4]:
# # # comment out
# # occurrence_df = (
# #     gbif_ecoregion_gdf
# #     # For each ecoregion, for each month...
# #     .groupby('ecoregion','month')
# #     # ...count the number of occurrences
# #     .agg(occurrences=('observation_id', 'count'))
# # )

# # chatgpt version - aha I was missing square brackets in syntax
# # Grouping data by ecoregion and month, and counting occurrences
# occurrence_df = (
#     gbif_ecoregion_gdf
#     # Group by both ecoregion and month
#     .groupby(['ecoregion', 'month'])
#     # Count the number of occurrences
#     .agg(occurrences=('observation_id', 'count'))
#     .reset_index()  # Resetting index to have a proper DataFrame
# )

# occurrence_df

#     # # Get rid of rare observations (possible misidentification?)
#     # occurrence_df = occurrence_df[...]

# # # Take the mean by ecoregion
# # mean_occurrences_by_ecoregion = (
# #     occurrence_df
# #     ...
# # )
# # # Take the mean by month
# # mean_occurrences_by_month = (
# #     occurrence_df
# #     ...
# # )

In [4]:
# Cleaning up the above two sections again fed to ChatGPT
# Performing the spatial join and resolving the ambiguous index
gbif_ecoregion_gdf = (
    eco_gdf
    # Match the CRS of the GBIF data and the ecoregions
    .to_crs(species_gbif_gdf.crs)
    # Find ecoregion for each observation using spatial join
    .sjoin(
        species_gbif_gdf,
        how='inner', 
        predicate='contains')
    # Select the required columns
    [['gbifID', 'OBJECTID', 'month', 'ECO_NAME']]
    # Reset index to handle potential index ambiguity
    .reset_index(drop=True)  # Drop the original index to avoid ambiguity
    # Rename columns for clarity
    .rename(columns={
        'OBJECTID': 'ecoregion',
        'gbifID': 'observation_id'})
)

# Grouping data by ecoregion and month, and counting occurrences
occurrence_df = (
    gbif_ecoregion_gdf
    # Group by both ecoregion and month
    .groupby(['ecoregion', 'month'])
    # Count the number of occurrences
    .agg(occurrences=('observation_id', 'count'))
    .reset_index()  # Resetting index to have a proper DataFrame
)

# occurrence_df
# has 350 rows
#now need to do this stuff
# Get rid of rare observations (possible misidentification?)
occurrence_df = occurrence_df[occurrence_df.occurrences>1]
#now has 308 rows getting rid of single observations in a region 
# (watch out for indent errors above did not work when indented!)
occurrence_df

# Take the mean by ecoregion
mean_occurrences_by_ecoregion = (
    occurrence_df
    .groupby('ecoregion')
    .mean()
)
# Take the mean by month
mean_occurrences_by_month = (
    occurrence_df
    .groupby('month')
    .mean()
)
mean_occurrences_by_month


,ecoregion,occurrences
month,,
1,206.666667,7.666667
2,323.666667,7.333333
3,481.666667,5.666667
4,403.527778,140.472222
5,378.406780,1207.169492
6,387.214286,1041.571429
7,366.157895,572.894737
8,390.216216,164.972973
9,367.384615,260.897436


### Normalize the observations

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Normalize</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Divide occurrences by the mean occurrences by month AND the mean
occurrences by ecoregion</li>
</ol></div></div>

In [11]:
# # Normalize by space and time for sampling effort  not working
# occurrence_df['norm_occurrences'] = (
#     occurrence_df
#     / mean_occurrences_by_ecoregion
#     / mean_occurrences_by_month
# )
# occurrence_df

In [5]:
# This is how ChatGPT took and cleaned up the code above 
# which appears to match the output in the video
# Take the mean by ecoregion
# but only run it once!
mean_occurrences_by_ecoregion = (
    occurrence_df
    .groupby('ecoregion')
    .agg(mean_occurrences_ecoregion=('occurrences', 'mean'))
    .reset_index()
)

# Take the mean by month
mean_occurrences_by_month = (
    occurrence_df
    .groupby('month')
    .agg(mean_occurrences_month=('occurrences', 'mean'))
    .reset_index()
)

# Merge the mean values back into the occurrence_df
occurrence_df = occurrence_df.merge(
    mean_occurrences_by_ecoregion,
    on='ecoregion',
    how='left'
)

occurrence_df = occurrence_df.merge(
    mean_occurrences_by_month,
    on='month',
    how='left'
)

# Normalize occurrences by ecoregion and month
occurrence_df['norm_occurrences'] = (
    occurrence_df['occurrences']
    / occurrence_df['mean_occurrences_ecoregion']
    / occurrence_df['mean_occurrences_month']
)

occurrence_df
#%store occurrence_df

,ecoregion,month,occurrences,mean_occurrences_ecoregion,mean_occurrences_month,norm_occurrences
0,13.0,5,2,2.000000,1207.169492,0.000828
1,13.0,6,2,2.000000,1041.571429,0.000960
2,13.0,7,2,2.000000,572.894737,0.001746
3,17.0,4,2,1425.333333,140.472222,0.000010
4,17.0,5,2980,1425.333333,1207.169492,0.001732
...,...,...,...,...,...,...
303,839.0,7,293,235.400000,572.894737,0.002173
304,839.0,8,40,235.400000,164.972973,0.001030
305,839.0,9,11,235.400000,260.897436,0.000179
306,845.0,9,25,16.000000,260.897436,0.005989


In [6]:
%store occurrence_df

Stored 'occurrence_df' (DataFrame)


<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><p>Make sure to store the new version of your <code>DataFrame</code> for
other notebooks!</p>
<div id="2e01613b" class="cell" data-execution_count="9">
<div class="sourceCode" id="cb1"><pre
class="sourceCode python cell-code"><code class="sourceCode python"><span id="cb1-1"><a href="#cb1-1" aria-hidden="true" tabindex="-1"></a><span class="op">%</span>store occurrence_df</span></code></pre></div>
</div></div></div>